# Etiquetage et annotation

Collecte de données automatisée de notre dossier d'images (116 images) pour le système de recommandation selon 5 critères dans un fichier 'etiquetage.json'.
- Orientation de l'images 
- Résolution de l'image
- ISO de l'image
- La catégorie : chien ou chat
- Animal dans la nature ou non ? proportion de vert dans l'image

In [3]:
from pandas import json_normalize
import pandas as pd
import json
import math
import matplotlib.pyplot as plot
import numpy as np

## EXIF des images

Dans un premier temps on récupère 3 données Exif dans un fichier JSON : nom de l'image, orientation de l'images, résolution de l'image et ISO de l'image

In [4]:
import os
import json
from PIL import Image
from PIL.ExifTags import TAGS

# Définir le dossier contenant les images
image_folder = "images/"
file = "etiquetage.json"

# Liste pour stocker les métadonnées
metadata_list = []

# Fonction pour déterminer l'orientation de l'image
def get_orientation(image):
    width, height = image.size
    if width > height:
        return "Paysage"
    elif width < height:
        return "Portrait"
    else:
        return "Carré"

# Parcourir tous les fichiers du dossier
for filename in os.listdir(image_folder):
    file_path = os.path.join(image_folder, filename)
    
    # Vérifier si c'est un fichier image
    try:
        with Image.open(file_path) as img:
            exif_data = img._getexif()  # Récupérer les métadonnées EXIF
            
            width, height = img.size  # Récupérer la résolution

            image_info = {
                "nom_image": filename,
                "orientation": get_orientation(img),
                "resolution": f"{width}x{height}",
                "iso": None
            }

            if exif_data:  
                for tag, value in exif_data.items():
                    tag_name = TAGS.get(tag, tag)  
                    
                    if tag_name == "ISOSpeedRatings":  
                        image_info["iso"] = value

            metadata_list.append(image_info)

    except Exception as e:
        print(f"Erreur avec {filename} : {e}")

# Sauvegarde des métadonnées dans un fichier JSON
with open(file, "w", encoding="utf-8") as json_file:
    json.dump(metadata_list, json_file, indent=4, ensure_ascii=False)

print(f"Métadonnées enregistrées dans {file}")


Métadonnées enregistrées dans etiquetage.json


# Catégorie : chien ou chat

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
import numpy as np
from PIL import Image

# Charger le modèle pré-entraîné
model = MobileNet(weights="imagenet")

# Charger une image et la préparer
image_path = "images/618736210a13.jpg"  # Remplace avec ton image
image = Image.open(image_path).resize((224, 224))
image_array = np.array(image)
image_array = np.expand_dims(image_array, axis=0)
image_array = preprocess_input(image_array)

# Faire la prédiction
predictions = model.predict(image_array)

# Décoder les résultats
decoded_predictions = tf.keras.applications.mobilenet.decode_predictions(predictions, top=3)
for i, (imagenet_id, label, score) in enumerate(decoded_predictions[0]):
    print(f"{i+1}: {label} ({score:.2f})")

## Couleurs

In [5]:
import json

file = "etiquetage.json"

# Charger le fichier JSON
with open(file, "r", encoding="utf-8") as json_file:
    data = json.load(json_file)

# Afficher le nombre d'éléments
print(f"Nombre d'éléments dans le fichier JSON : {len(data)}")


Nombre d'éléments dans le fichier JSON : 116
